# 🌱 Agricultural Data Analysis & Model Training

This notebook integrates the **Soil Nutrient Analysis** and **Medicine Recommendation** modules.
It performs data loading, preprocessing, model training, and evaluation for both systems.

In [4]:
import pandas as pd
import numpy as np
import joblib
import os
import json
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error, r2_score, confusion_matrix

try:
    from xgboost import XGBClassifier, XGBRegressor
except ImportError: 
    print("XGBoost not installed.")
try:
    from lightgbm import LGBMClassifier, LGBMRegressor
except ImportError:
    print("LightGBM not installed.")
try:
    from catboost import CatBoostClassifier, CatBoostRegressor
except ImportError:
    print("CatBoost not installed.")

warnings.filterwarnings('ignore')

# Constants
DATA_DIR = r"D:\SIH\NITR\data2"
SOIL_DATA_PATH = os.path.join(DATA_DIR, "NIR spectra Data of Soil samples and fertility properties as matrix X and Y.xlsx")
MEDICINE_DATA_PATH = os.path.join(DATA_DIR, "medicine", "medicine_data.csv")
MODEL_DIR = os.path.join(DATA_DIR, "models")
os.makedirs(MODEL_DIR, exist_ok=True)

# ---------------------------------------------------------
# 1. Soil Analysis Module
# ---------------------------------------------------------

def load_soil_data(file_path):
    print(f"Loading soil data from {file_path}...")
    try:
        df_x = pd.read_excel(file_path, sheet_name='Data X (spectra data)')
        df_y = pd.read_excel(file_path, sheet_name='Data Y (soil properties)')
    except Exception as e:
        raise ValueError(f"Error loading sheets: {e}")

    if 'No.' in df_y.columns:
        df_y.rename(columns={'No.': 'Num of soil Sample'}, inplace=True)

    merged_df = pd.merge(df_x, df_y, on='Num of soil Sample', how='inner')
    
    # Identify Targets
    target_cols = df_y.columns.tolist()
    if 'Num of soil Sample' in target_cols: target_cols.remove('Num of soil Sample')
    
    # Handle Missing Nitrogen (Derive from OC if needed)
    if not any('nitrogen' in c.lower() or c.strip() == 'N' for c in target_cols):
        oc_col = next((c for c in target_cols if 'oc' in c.lower() or 'organic carbon' in c.lower()), None)
        if oc_col:
            print(f"Deriving 'Nitrogen' from '{oc_col}'")
            merged_df['Nitrogen'] = merged_df[oc_col] / 10.0
            target_cols.append('Nitrogen')
            
    return merged_df, target_cols

def bin_nutrient(value, percentiles):
    if value < percentiles[0]: return 'Low'
    elif value < percentiles[1]: return 'Medium'
    else: return 'High'

def preprocess_soil_data(df, target_cols):
    df = df.copy().fillna(df.mean(numeric_only=True))
    
    # Features vs Targets
    feature_cols = [c for c in df.columns if c not in target_cols and c != 'Num of soil Sample']
    
    # --- Fertility Score Calculation ---
    targets_numeric = df[target_cols].select_dtypes(include=[np.number])
    targets_norm = (targets_numeric - targets_numeric.min()) / (targets_numeric.max() - targets_numeric.min())
    
    df['Soil_Fertility_Score'] = 0.0
    components = 0
    
    # Try to find N, P, K columns for scoring
    n_col = next((c for c in targets_norm.columns if 'nitrogen' in c.lower()), None)
    if not n_col: n_col = next((c for c in targets_norm.columns if 'oc' in c.lower()), None)
        
    p_col = next((c for c in targets_norm.columns if 'P' in c or 'phosphorus' in c.lower()), None)
    k_col = next((c for c in targets_norm.columns if 'K' in c or 'potassium' in c.lower()), None)

    if n_col: 
        df['Soil_Fertility_Score'] += targets_norm[n_col] * 0.4; components += 1
    if p_col: 
        df['Soil_Fertility_Score'] += targets_norm[p_col] * 0.3; components += 1
    if k_col: 
        df['Soil_Fertility_Score'] += targets_norm[k_col] * 0.3; components += 1
    
    if components > 0:
        df['Soil_Fertility_Score'] *= 100
        
    # --- Binning & Encoding ---
    binned_targets = {}
    encoders = {}
    
    cols_to_bin = [c for c in target_cols if any(x in c.lower() for x in ['nitrogen', 'n', 'phosphorus', 'p', 'potassium', 'k']) and 'ph' not in c.lower()]
    
    for col in cols_to_bin:
        if col not in df.columns: continue
        percentiles = np.percentile(df[col], [33, 66])
        new_col = f"{col}_Class"
        df[new_col] = df[col].apply(lambda x: bin_nutrient(x, percentiles))
        
        le = LabelEncoder()
        df[new_col] = le.fit_transform(df[new_col])
        binned_targets[col] = new_col
        encoders[col] = le
        print(f"Encoded {col} -> classes: {le.classes_}")

    # --- Scaling ---
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(df[feature_cols]), columns=feature_cols)
    
    return X, df, binned_targets, encoders, scaler

def train_soil_models(X, y, task_type='classification'):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    models = {}
    if task_type == 'classification':
        models['RandomForest'] = RandomForestClassifier(random_state=42)
        if 'XGBClassifier' in globals(): models['XGBoost'] = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
        if 'LGBMClassifier' in globals(): models['LightGBM'] = LGBMClassifier(random_state=42, verbose=-1, verbosity=-1)
        if 'CatBoostClassifier' in globals(): models['CatBoost'] = CatBoostClassifier(verbose=0, random_state=42)
    else:
        models['RandomForest'] = RandomForestRegressor(random_state=42)
        if 'XGBRegressor' in globals(): models['XGBoost'] = XGBRegressor(random_state=42)
        if 'LGBMRegressor' in globals(): models['LightGBM'] = LGBMRegressor(random_state=42, verbose=-1, verbosity=-1)
        if 'CatBoostRegressor' in globals(): models['CatBoost'] = CatBoostRegressor(verbose=0, random_state=42)

    best_score = -np.inf
    best_model = None
    best_name = ""
    
    results = {}

    for name, model in models.items():
        try:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            
            if task_type == 'classification':
                score = f1_score(y_test, y_pred, average='weighted', zero_division=0)
                acc = accuracy_score(y_test, y_pred)
                results[name] = {'F1': score, 'Accuracy': acc}
                print(f"  {name}: F1={score:.4f} Acc={acc:.4f}")
            else:
                score = r2_score(y_test, y_pred)
                mse = mean_squared_error(y_test, y_pred)
                results[name] = {'R2': score, 'MSE': mse}
                print(f"  {name}: R2={score:.4f} MSE={mse:.4f}")
                
            if score > best_score:
                best_score = score
                best_model = model
                best_name = name
        except Exception as e:
            print(f"Failed to train {name}: {e}")
            
    return best_model, best_name, results

# ---------------------------------------------------------
# 2. Medicine Recommendation Module
# ---------------------------------------------------------

def run_medicine_pipeline():
    if os.path.exists(MEDICINE_DATA_PATH):
        print(f"\nLoading medicine data from {MEDICINE_DATA_PATH}...")
        df_med = pd.read_csv(MEDICINE_DATA_PATH)
        
        # Encoding
        le_crop = LabelEncoder()
        le_disease = LabelEncoder()
        le_medicine = LabelEncoder()
        
        df_med['crop_n'] = le_crop.fit_transform(df_med['Crop'])
        df_med['disease_n'] = le_disease.fit_transform(df_med['Disease'])
        y_med = le_medicine.fit_transform(df_med['Medicine Name'])
        
        X_med = df_med[['crop_n', 'disease_n']]
        
        # Scaling
        scaler_med = StandardScaler()
        X_med_scaled = scaler_med.fit_transform(X_med)
        
        # Training
        print("Training Medicine Recommendation Model...")
        med_model = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, random_state=42)
        med_model.fit(X_med_scaled, y_med)
        
        # Validation
        acc = med_model.score(X_med_scaled, y_med)
        print(f"Model Accuracy: {acc*100:.2f}%")
        
        # Save
        joblib.dump(med_model, os.path.join(MODEL_DIR, 'medicine_model.joblib'))
        joblib.dump(scaler_med, os.path.join(MODEL_DIR, 'medicine_scaler.joblib'))
        joblib.dump(le_crop, os.path.join(MODEL_DIR, 'le_crop.joblib'))
        joblib.dump(le_disease, os.path.join(MODEL_DIR, 'le_disease.joblib'))
        joblib.dump(le_medicine, os.path.join(MODEL_DIR, 'le_medicine.joblib'))
        print("Medicine Models Saved.")
        
    else:
        print("Medicine data file not found.")

# ---------------------------------------------------------
# Main Execution
# ---------------------------------------------------------

if __name__ == "__main__":
    print("--- Starting Agricultural Data Analysis ---\n")
    
    # 1. Run Soil Analysis
    try:
        df_soil, soil_targets = load_soil_data(SOIL_DATA_PATH)
        print(f"Soil Data Loaded. Shape: {df_soil.shape}")
        
        X_soil, df_soil_proc, soil_classes, soil_encoders, soil_scaler = preprocess_soil_data(df_soil, soil_targets)
        
        # Train Classification Models
        for original_col, class_col in soil_classes.items():
            print(f"\nTraining for {original_col}...")
            y = df_soil_proc[class_col]
            best_model, name, res = train_soil_models(X_soil, y, 'classification')
            print(f">> Best Model for {original_col}: {name}")
            
            joblib.dump(best_model, os.path.join(MODEL_DIR, f"BEST_{original_col.split()[0]}.joblib"))
        
        # Train Regression Model (Fertility Score)
        print("\nTraining for Soil Fertility Score...")
        y_score = df_soil_proc['Soil_Fertility_Score']
        best_model_s, name_s, res_s = train_soil_models(X_soil, y_score, 'regression')
        print(f">> Best Model for Fertility Score: {name_s}")
        joblib.dump(best_model_s, os.path.join(MODEL_DIR, "BEST_FertilityScore.joblib"))
        
    except Exception as e:
        print(f"CRITICAL ERROR in Soil Analysis: {e}")
        import traceback
        traceback.print_exc()

    # 2. Run Medicine Analysis
    try:
        run_medicine_pipeline()
    except Exception as e:
        print(f"CRITICAL ERROR in Medicine Analysis: {e}")

    print("\n--- Analysis Complete ---")


--- Starting Agricultural Data Analysis ---

Loading soil data from D:\SIH\NITR\data2\NIR spectra Data of Soil samples and fertility properties as matrix X and Y.xlsx...
Soil Data Loaded. Shape: (40, 1564)
Encoded  N content (%) -> classes: ['High' 'Low' 'Medium']
Encoded P (ppm) -> classes: ['High' 'Low' 'Medium']
Encoded K (cmol/kg) -> classes: ['High' 'Low' 'Medium']
Encoded Mg (cmol/kg) -> classes: ['High' 'Low' 'Medium']
Encoded Ca (cmol/kg) -> classes: ['High' 'Low' 'Medium']

Training for  N content (%)...
  RandomForest: F1=0.5208 Acc=0.5000
  XGBoost: F1=0.5938 Acc=0.6250
  LightGBM: F1=0.2045 Acc=0.3750
  CatBoost: F1=0.7500 Acc=0.7500
>> Best Model for  N content (%): CatBoost

Training for P (ppm)...
  RandomForest: F1=0.5107 Acc=0.5000
  XGBoost: F1=0.3750 Acc=0.3750
  LightGBM: F1=0.1000 Acc=0.2500
  CatBoost: F1=0.5107 Acc=0.5000
>> Best Model for P (ppm): RandomForest

Training for K (cmol/kg)...
  RandomForest: F1=0.3690 Acc=0.3750
  XGBoost: F1=0.3917 Acc=0.3750
  Lig

In [5]:
import pandas as pd
import numpy as np
import joblib
import os
import json
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error, r2_score, confusion_matrix

try:
    from xgboost import XGBClassifier, XGBRegressor
except ImportError: 
    print("XGBoost not installed.")
try:
    from lightgbm import LGBMClassifier, LGBMRegressor
except ImportError:
    print("LightGBM not installed.")
try:
    from catboost import CatBoostClassifier, CatBoostRegressor
except ImportError:
    print("CatBoost not installed.")

warnings.filterwarnings('ignore')

# Constants
DATA_DIR = r"D:\SIH\NITR\data2"
SOIL_DATA_PATH = os.path.join(DATA_DIR, "NIR spectra Data of Soil samples and fertility properties as matrix X and Y.xlsx")
MEDICINE_DATA_PATH = os.path.join(DATA_DIR, "medicine", "medicine_data.csv")
MODEL_DIR = os.path.join(DATA_DIR, "models")
os.makedirs(MODEL_DIR, exist_ok=True)

# ---------------------------------------------------------
# 1. Soil Analysis Module
# ---------------------------------------------------------

def load_soil_data(file_path):
    print(f"Loading soil data from {file_path}...")
    try:
        df_x = pd.read_excel(file_path, sheet_name='Data X (spectra data)')
        df_y = pd.read_excel(file_path, sheet_name='Data Y (soil properties)')
    except Exception as e:
        raise ValueError(f"Error loading sheets: {e}")

    if 'No.' in df_y.columns:
        df_y.rename(columns={'No.': 'Num of soil Sample'}, inplace=True)

    merged_df = pd.merge(df_x, df_y, on='Num of soil Sample', how='inner')
    
    # Identify Targets
    target_cols = df_y.columns.tolist()
    if 'Num of soil Sample' in target_cols: target_cols.remove('Num of soil Sample')
    
    # Handle Missing Nitrogen (Derive from OC if needed)
    if not any('nitrogen' in c.lower() or c.strip() == 'N' for c in target_cols):
        oc_col = next((c for c in target_cols if 'oc' in c.lower() or 'organic carbon' in c.lower()), None)
        if oc_col:
            print(f"Deriving 'Nitrogen' from '{oc_col}'")
            merged_df['Nitrogen'] = merged_df[oc_col] / 10.0
            target_cols.append('Nitrogen')
            
    return merged_df, target_cols

def bin_nutrient(value, percentiles):
    if value < percentiles[0]: return 'Low'
    elif value < percentiles[1]: return 'Medium'
    else: return 'High'

def preprocess_soil_data(df, target_cols):
    df = df.copy().fillna(df.mean(numeric_only=True))
    
    # Features vs Targets
    feature_cols = [c for c in df.columns if c not in target_cols and c != 'Num of soil Sample']
    
    # --- Fertility Score Calculation ---
    targets_numeric = df[target_cols].select_dtypes(include=[np.number])
    targets_norm = (targets_numeric - targets_numeric.min()) / (targets_numeric.max() - targets_numeric.min())
    
    df['Soil_Fertility_Score'] = 0.0
    components = 0
    
    # Try to find N, P, K columns for scoring
    n_col = next((c for c in targets_norm.columns if 'nitrogen' in c.lower()), None)
    if not n_col: n_col = next((c for c in targets_norm.columns if 'oc' in c.lower()), None)
        
    p_col = next((c for c in targets_norm.columns if 'P' in c or 'phosphorus' in c.lower()), None)
    k_col = next((c for c in targets_norm.columns if 'K' in c or 'potassium' in c.lower()), None)

    if n_col: 
        df['Soil_Fertility_Score'] += targets_norm[n_col] * 0.4; components += 1
    if p_col: 
        df['Soil_Fertility_Score'] += targets_norm[p_col] * 0.3; components += 1
    if k_col: 
        df['Soil_Fertility_Score'] += targets_norm[k_col] * 0.3; components += 1
    
    if components > 0:
        df['Soil_Fertility_Score'] *= 100
        
    # --- Binning & Encoding ---
    binned_targets = {}
    encoders = {}
    
    cols_to_bin = [c for c in target_cols if any(x in c.lower() for x in ['nitrogen', 'n', 'phosphorus', 'p', 'potassium', 'k']) and 'ph' not in c.lower()]
    
    for col in cols_to_bin:
        if col not in df.columns: continue
        percentiles = np.percentile(df[col], [33, 66])
        new_col = f"{col}_Class"
        df[new_col] = df[col].apply(lambda x: bin_nutrient(x, percentiles))
        
        le = LabelEncoder()
        df[new_col] = le.fit_transform(df[new_col])
        binned_targets[col] = new_col
        encoders[col] = le
        print(f"Encoded {col} -> classes: {le.classes_}")

    # --- Scaling ---
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(df[feature_cols]), columns=feature_cols)
    
    return X, df, binned_targets, encoders, scaler

def train_soil_models(X, y, task_type='classification'):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    models = {}
    if task_type == 'classification':
        models['RandomForest'] = RandomForestClassifier(random_state=42)
        models['GradientBoosting'] = GradientBoostingClassifier(random_state=42)
        models['SVM'] = SVC(kernel='rbf', probability=True, random_state=42)
        models['KNN'] = KNeighborsClassifier(n_neighbors=5)
        models['MLP_NeuralNet'] = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
        
        if 'XGBClassifier' in globals(): models['XGBoost'] = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
        if 'LGBMClassifier' in globals(): models['LightGBM'] = LGBMClassifier(random_state=42, verbose=-1, verbosity=-1)
        if 'CatBoostClassifier' in globals(): models['CatBoost'] = CatBoostClassifier(verbose=0, random_state=42)
    else:
        models['RandomForest'] = RandomForestRegressor(random_state=42)
        models['GradientBoosting'] = GradientBoostingRegressor(random_state=42)
        models['SVR'] = SVR(kernel='rbf')
        models['KNN'] = KNeighborsRegressor(n_neighbors=5)
        models['MLP_NeuralNet'] = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
        models['LinearReg'] = LinearRegression()
        
        if 'XGBRegressor' in globals(): models['XGBoost'] = XGBRegressor(random_state=42)
        if 'LGBMRegressor' in globals(): models['LightGBM'] = LGBMRegressor(random_state=42, verbose=-1, verbosity=-1)
        if 'CatBoostRegressor' in globals(): models['CatBoost'] = CatBoostRegressor(verbose=0, random_state=42)

    best_score = -np.inf
    best_model = None
    best_name = ""
    
    results = {}

    for name, model in models.items():
        try:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            
            if task_type == 'classification':
                score = f1_score(y_test, y_pred, average='weighted', zero_division=0)
                acc = accuracy_score(y_test, y_pred)
                results[name] = {'F1': score, 'Accuracy': acc}
                print(f"  {name}: F1={score:.4f} Acc={acc:.4f}")
            else:
                score = r2_score(y_test, y_pred)
                mse = mean_squared_error(y_test, y_pred)
                results[name] = {'R2': score, 'MSE': mse}
                print(f"  {name}: R2={score:.4f} MSE={mse:.4f}")
                
            if score > best_score:
                best_score = score
                best_model = model
                best_name = name
        except Exception as e:
            print(f"  {name} Failed: {e}")
            
    return best_model, best_name, results

# ---------------------------------------------------------
# 2. Medicine Recommendation Module
# ---------------------------------------------------------

def run_medicine_pipeline():
    if os.path.exists(MEDICINE_DATA_PATH):
        print(f"\nLoading medicine data from {MEDICINE_DATA_PATH}...")
        df_med = pd.read_csv(MEDICINE_DATA_PATH)
        
        # Encoding
        le_crop = LabelEncoder()
        le_disease = LabelEncoder()
        le_medicine = LabelEncoder()
        
        df_med['crop_n'] = le_crop.fit_transform(df_med['Crop'])
        df_med['disease_n'] = le_disease.fit_transform(df_med['Disease'])
        y_med = le_medicine.fit_transform(df_med['Medicine Name'])
        
        X_med = df_med[['crop_n', 'disease_n']]
        
        # Scaling
        scaler_med = StandardScaler()
        X_med_scaled = scaler_med.fit_transform(X_med)
        
        # Split
        X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_med_scaled, y_med, test_size=0.2, random_state=42)
        
        # Training comparison
        print("Training comparison for Medicine Recommendation...")
        med_models = {
            'SGD': SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, random_state=42),
            'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
            'SVM': SVC(kernel='rbf', probability=True, random_state=42),
            'KNN': KNeighborsClassifier(n_neighbors=5),
            'MLP': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
        }
        
        if 'XGBClassifier' in globals(): 
            med_models['XGBoost'] = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

        best_med_acc = 0
        best_med_model = None
        best_med_name = ""

        for name, model in med_models.items():
            try:
                model.fit(X_train_m, y_train_m)
                acc = model.score(X_test_m, y_test_m)
                print(f"  {name} Accuracy: {acc*100:.2f}%")
                
                if acc > best_med_acc:
                    best_med_acc = acc
                    best_med_model = model
                    best_med_name = name
            except Exception as e:
                print(f"  {name} Failed: {e}")
        
        print(f">> Best Medicine Model: {best_med_name} ({best_med_acc*100:.2f}%)")
        
        # Save Best
        joblib.dump(best_med_model, os.path.join(MODEL_DIR, 'medicine_model.joblib'))
        joblib.dump(scaler_med, os.path.join(MODEL_DIR, 'medicine_scaler.joblib'))
        joblib.dump(le_crop, os.path.join(MODEL_DIR, 'le_crop.joblib'))
        joblib.dump(le_disease, os.path.join(MODEL_DIR, 'le_disease.joblib'))
        joblib.dump(le_medicine, os.path.join(MODEL_DIR, 'le_medicine.joblib'))
        print("Best Medicine Models Saved.")
        
    else:
        print("Medicine data file not found.")

# ---------------------------------------------------------
# Main Execution
# ---------------------------------------------------------

if __name__ == "__main__":
    print("--- Starting Agricultural Data Analysis (Expanded) ---\n")
    
    # 1. Run Soil Analysis
    try:
        df_soil, soil_targets = load_soil_data(SOIL_DATA_PATH)
        print(f"Soil Data Loaded. Shape: {df_soil.shape}")
        
        X_soil, df_soil_proc, soil_classes, soil_encoders, soil_scaler = preprocess_soil_data(df_soil, soil_targets)
        
        # Train Classification Models
        for original_col, class_col in soil_classes.items():
            print(f"\nTraining for {original_col}...")
            y = df_soil_proc[class_col]
            best_model, name, res = train_soil_models(X_soil, y, 'classification')
            print(f">> Best Model for {original_col}: {name}")
            
            joblib.dump(best_model, os.path.join(MODEL_DIR, f"BEST_{original_col.split()[0]}.joblib"))
        
        # Train Regression Model (Fertility Score)
        print("\nTraining for Soil Fertility Score...")
        y_score = df_soil_proc['Soil_Fertility_Score']
        best_model_s, name_s, res_s = train_soil_models(X_soil, y_score, 'regression')
        print(f">> Best Model for Fertility Score: {name_s}")
        joblib.dump(best_model_s, os.path.join(MODEL_DIR, "BEST_FertilityScore.joblib"))
        
    except Exception as e:
        print(f"CRITICAL ERROR in Soil Analysis: {e}")
        import traceback
        traceback.print_exc()

    # 2. Run Medicine Analysis
    try:
        run_medicine_pipeline()
    except Exception as e:
        print(f"CRITICAL ERROR in Medicine Analysis: {e}")

    print("\n--- Analysis Complete ---")


--- Starting Agricultural Data Analysis (Expanded) ---

Loading soil data from D:\SIH\NITR\data2\NIR spectra Data of Soil samples and fertility properties as matrix X and Y.xlsx...
Soil Data Loaded. Shape: (40, 1564)
Encoded  N content (%) -> classes: ['High' 'Low' 'Medium']
Encoded P (ppm) -> classes: ['High' 'Low' 'Medium']
Encoded K (cmol/kg) -> classes: ['High' 'Low' 'Medium']
Encoded Mg (cmol/kg) -> classes: ['High' 'Low' 'Medium']
Encoded Ca (cmol/kg) -> classes: ['High' 'Low' 'Medium']

Training for  N content (%)...
  RandomForest: F1=0.5208 Acc=0.5000
  GradientBoosting: F1=0.6042 Acc=0.6250
  SVM: F1=0.6500 Acc=0.6250
  KNN: F1=0.7500 Acc=0.7500
  MLP_NeuralNet: F1=0.7292 Acc=0.7500
  XGBoost: F1=0.5938 Acc=0.6250
  LightGBM: F1=0.2045 Acc=0.3750
  CatBoost: F1=0.7500 Acc=0.7500
>> Best Model for  N content (%): KNN

Training for P (ppm)...
  RandomForest: F1=0.5107 Acc=0.5000
  GradientBoosting: F1=0.5000 Acc=0.5000
  SVM: F1=0.5107 Acc=0.5000
  KNN: F1=0.4167 Acc=0.5000
  M

# 💊 Module 2: Medicine Recommendation System

Predicts appropriate medicine based on Crop and Disease.

In [ ]:
if os.path.exists(MEDICINE_DATA_PATH):
    print(f"Loading medicine data from {MEDICINE_DATA_PATH}...")
    df_med = pd.read_csv(MEDICINE_DATA_PATH)
    
    # Encoding
    le_crop = LabelEncoder()
    le_disease = LabelEncoder()
    le_medicine = LabelEncoder()
    
    df_med['crop_n'] = le_crop.fit_transform(df_med['Crop'])
    df_med['disease_n'] = le_disease.fit_transform(df_med['Disease'])
    y_med = le_medicine.fit_transform(df_med['Medicine Name'])
    
    X_med = df_med[['crop_n', 'disease_n']]
    
    # Scaling
    scaler_med = StandardScaler()
    X_med_scaled = scaler_med.fit_transform(X_med)
    
    # Training
    print("Training Medicine Recommendation Model...")
    med_model = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, random_state=42)
    med_model.fit(X_med_scaled, y_med)
    
    # Validation
    acc = med_model.score(X_med_scaled, y_med)
    print(f"Model Accuracy: {acc*100:.2f}%")
    
    # Save
    joblib.dump(med_model, os.path.join(MODEL_DIR, 'medicine_model.joblib'))
    joblib.dump(scaler_med, os.path.join(MODEL_DIR, 'medicine_scaler.joblib'))
    joblib.dump(le_crop, os.path.join(MODEL_DIR, 'le_crop.joblib'))
    joblib.dump(le_disease, os.path.join(MODEL_DIR, 'le_disease.joblib'))
    joblib.dump(le_medicine, os.path.join(MODEL_DIR, 'le_medicine.joblib'))
    print("Medicine Models Saved.")
    
else:
    print("Medicine data file not found.")

## Summary
All models have been trained and saved to the `models/` directory.